### **Load Environment variables from .env file**

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
from IPython.display import display, HTML
import os

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME  = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")

SQL_SERVER = os.getenv("SQL_SERVER")
SQL_USER = os.getenv("SQL_USER")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DBNAME = os.getenv("SQL_DBNAME")

In [3]:
def init_llm(model=OPENAI_GPT35_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT35_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

llm = init_llm()

In [4]:
embeddings = AzureOpenAIEmbeddings(
    model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    openai_api_version="2024-02-15-preview",
    chunk_size = 1
    )


#### **Run ONLY ONCE to create the embeddings - Split text into chunks** 

In [5]:
fileName = "./data/fabric-data-engineering.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print("Number of pages: ", len(pages))

Number of pages:  292


#### **Run ONLY ONCE to create the embeddings - Create embeddings and save to FAISS**

In [9]:
db = FAISS.from_documents(documents=pages, embedding=embeddings)
# save the FAISS index to disk
db.save_local("./dbs/documentation/faiss_index")

#### Initialize retrieval API WITH your data

In [13]:
# load the vector store to memory
vectorStore = FAISS.load_local("./dbs/documentation/faiss_index", embeddings)
retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 2})  # returns 2 most similar vectors/documents
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

#### **Ask questions**

In [16]:
r = qa.invoke({"query": "Can I use PowerBI datamart for 110 GB data volume? Explain which other options are available for data volumes of 110 GB?"})

display(HTML(r['result']))

In [20]:
for document in r['source_documents']:
    display(HTML(document.page_content))

In [21]:
r = qa.invoke({"query": "What are the steps to load a CSV file to a delta table in Microsoft Fabric?"})

display(HTML(r['result']))

In [22]:
for document in r['source_documents']:
    display(HTML(document.page_content))